In [ ]:
using ITensors, ITensorMPS, Plots

function run_dmrg_ssh(N::Int, t1::Float64, t2::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Vector{Float64})
    # Define fermionic sites
    sites = siteinds("Fermion", N)

    # Construct SSH Hamiltonian as an MPO
    os = OpSum()
    for j in 1:N-1
        hopping = j % 2 == 1 ? t1 : t2
        os += -hopping, "Cdag", j, "C", j + 1
        os += -hopping, "Cdag", j + 1, "C", j
    end

    os += -t2, "Cdag", 1, "C", N
    os += -t2, "Cdag", N, "C", 1

    H = MPO(os, sites)


    # Compute the ground state psi0
    state = [isodd(n) ? "1" : "0" for n in 1:N]  # Alternating filling pattern
    psi0_init = randomMPS(sites,state)
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)


    # Compute the first excited state psi1 
    psi1_init = randomMPS(sites,state)  
    energy_excited, psi_excited = dmrg(H, [psi_ground], psi1_init; nsweeps, maxdim, cutoff, noise, weight=weight)


    #copute the second excited state psi2
    #psi2_init = random_mps(sites;linkdims=2)
    #energy_2excited,psi_2excited = dmrg(H,[psi_ground,psi_excited],psi2_init;nsweeps,maxdim,cutoff,noise,weight=0.8)
  



    energy_gap = energy_excited - energy_ground

    println("Ground state energy: ", energy_ground)
    println("Excited state energy: ", energy_excited)
    #println("2nd Excited state energy: ", energy_2excited)
    @show inner(psi_excited,psi_ground)


    return energy_ground, psi_ground, energy_excited, psi_excited, energy_gap
end

# Parameters
N = 30           # Number of sites
t1 = 1.0       
h = 4.0
weight = 2 
noise = [1E-10]
#t2_values=[1.6]
t2_values = collect(0.5:0.10:1.6)  # Varying t2 from 0.5 to 1.6
nsweeps = 5      # Number of DMRG sweeps
maxdim = [20, 40, 50, 100, 200] 
cutoff = [1E-10] 
# Compute energy gaps for different t2 values
energy_gaps = []
for t2 in t2_values
    _, _, _, _, energy_gap = run_dmrg_ssh(N, t1, t2, nsweeps, maxdim, cutoff)
    push!(energy_gaps, energy_gap)
end

# Plot energy gap variation
plot(t2_values, energy_gaps, xlabel="t2", ylabel="Energy Gap", title="Energy Gap vs t2", marker=:o, linewidth=2)
display(plot)
savefig("ssh_engap_plot.png")
println("Energy gap values: ", energy_gaps)
